In [1]:
# Scrape LinkedIn companies based on user inputted search query and update into Company Web Scrape Google Sheets 
# Script for LinkedIn Company Mining Feature

#import libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [2]:
#Global variables
query = ''
queries = []
companies = []
numberofPages = 0
cellLocationQuery = 'C2'
cellLocationPages = 'B2'
sheetIndexRead = 0
sheetIndexWrite = 2
columnName = 'Company Names (Leave Blank if Using LinkedIn Company Mining)'
password = ''
email = 'zach.jl.wong@gmail.com'

#paths/urls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
loginurl = 'https://www.linkedin.com/uas/login'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoLinkedInFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
pathtoPassword = '/Users/zacharywong/Documents/LinkedIn/LinkedIn-Password-Secret.txt'

#waitTimes
waitLinkedInTime = 10
waitCaptcha = 25
waitSearch= 1
waitLogin = 0.8


# xPaths, classes, and javascripts
searchBarXPath = '//*[@id="global-nav-typeahead"]/input'
companyFilterXPath = '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button'
scrollDownScript = "window.scrollTo(0,document.body.scrollHeight)"
pageButtonClass = "button[type='button']"
companyTitleClass = 'app-aware-link'
companyTitleAttr = 'a'

In [3]:
# helper function to read in value from spreadsheet
def readinValue(cellLocation, sheetIndexRead):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexRead)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

def readQueries():
    global queries
    row = 2;
    sheetIndex = 0
    isDone = False
    cellLocationColumn = 'C'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            queries.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    queries = queries[0:len(queries)-1]
    print('Queries: ', str(queries))
    
# read in query from spreadsheet
def readinQuery(queryIndex):
    global query
    query = queries[queryIndex]
    print('currentQuery: ' + query)
    
    
# read in number of pages from spreadsheet
def readinPages():
    global numberofPages
    numberofPages = int(readinValue(cellLocationPages, sheetIndexRead))
    print('numberofPages: ', numberofPages)
    
    
    
# login to linkedin 
def login(driver):
    driver.get(loginurl)
    emailInput = driver.find_element(By.ID, 'username')
    emailInput.send_keys(email)
    time.sleep(waitLogin)
    passwordInput = driver.find_element(By.ID, 'password')
    with open (pathtoPassword, 'r') as file:
        global password
        password = file.read()
    passwordInput.send_keys(password)
    time.sleep(waitLogin)
    passwordInput.send_keys(Keys.RETURN)
    
    
# search query on LinkedIn
def searchLinkedIn(driver):
    #search query
    try:
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
    except: 
        time.sleep(waitCaptcha)
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
        
        
# Click Company Filter option on LinkedIn
def companyFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, companyFilterXPath)))
    companyButtons = driver.find_elements_by_css_selector(pageButtonClass)
    for button in companyButtons: 
        buttonText = button.text
        if (buttonText == 'Companies'): 
            button.click()
            return;
# Click onto next page 


# Click onto next page 
def nextPage(currentPage, driver, previousPage):
    nextPageSuccess = False; 
    driver.execute_script(scrollDownScript)
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector(pageButtonClass)
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber) 
        if (buttonNumber == str(currentPage) and currentPage != previousPage):
            #print('hello')
            page.click()
            nextPageSuccess = True; 
            break
    if (nextPageSuccess == False):
        for page in pages:
            buttonNumber = page.text
            if (buttonNumber == '…'):
                nextPageSuccess = True 
                print("click more option")
                page.click()
                return
        if nextPageSuccess == False:
            print("No more results")
            return nextPageSuccess
        
            
            
# add companies into a pandas dataframe 
def addCompanies(driver):
    global companies
    previousPage = 0
    currentPage = 1
    while (currentPage <= numberofPages):
        time.sleep(2)
        soup = bs(driver.page_source, 'html.parser')
        companySearch = soup.find_all(companyTitleAttr, class_ = companyTitleClass)
        for company in companySearch:
            companyName = company.text
            if '\n\n\n' not in companyName:
                if (companyName.replace('\n', '')) not in companies:
                    companies.append(companyName.replace('\n', ''))
        previousPage = currentPage
        currentPage += 1
        nextPageSuccess = nextPage(currentPage, driver, previousPage)
        if (nextPageSuccess==False):
            return companies 
    return companies


# export final list of companies to CSV 
def exportCSV(companies):
    df = pd.DataFrame(companies, columns = [columnName])
    df.to_csv(pathtoLinkedInFile)
    return df


# Update spreadsheet with list of companies
def updateSpreadSheet(df, sheetIndexWrite):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexWrite)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
    
# run all functions to scrape LinkedIn companies  
def runLinkedIn(queryIndex):
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    readinQuery(queryIndex)
    readinPages()
    login(driver)
    searchLinkedIn(driver)
    companyFilter(driver)
    companies = addCompanies(driver)
    

In [4]:
# run program 
startTime = time.time()
readQueries()
print('Queries: ', queries)
queryIndex = 0
while queryIndex < len(queries):
    runLinkedIn(queryIndex)
    print(companies)
    queryIndex += 1
df = exportCSV(companies)
updateSpreadSheet(df, sheetIndexWrite)
endTime = time.time()
elapsedTime = endTime - startTime
print('Time Elapsed: ', elapsedTime)
df

Queries:  ['mental health chatbot', 'mental health artificial intelligence ']
Queries:  ['mental health chatbot', 'mental health artificial intelligence ']
currentQuery: mental health chatbot
numberofPages:  20


/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_18395/3004049052.py:81: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  companyButtons = driver.find_elements_by_css_selector(pageButtonClass)
/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_18395/3004049052.py:96: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pages = driver.find_elements_by_css_selector(pageButtonClass)


No more results
['Woebot Health', 'Wysa', 'Plutus Health Inc.', "Julian Jewel's AI Bot", 'X2AI', 'Moodcare', 'Edumiam', 'Centre for Digital Health Interventions', 'togetherAI', 'Etsimo Healthcare', 'Kibo', 'Olive Health Technology LLC', 'Calmsie', 'Enovation Zaurus', 'Aliae', 'Rdentify ', 'Curis AI Inc.', 'Wazi', 'Wellcare', 'OneSeventeen Media, PBC', 'Well Good', 'EMHware Software Inc.', 'Kvell.App', 'Eating Disorders In Student Services', 'Wholistics.Health', 'Fan Health Network', 'Mr Young', 'CovidFYI', 'Life Clips, Inc. ', 'Cognint', 'mhapy (Mental Health Accountability Project)', 'Echo', 'BOTSITIVITY', 'FriendTales', 'eMara inc.', 'ibindo GmbH', 'Ecstatic', 'LunaShift', 'Reva', 'The Wellness Connection Copywriting (TWCC)', 'Mindboost', 'Project Kitchen Table', 'Robology', 'TotemX Labs', "Children's Angels Project", 'TechTree', 'comentis GmbH', 'mindnim', 'NafsiTalk', 'WellBeings', 'SumYin', 'Mighty Me', 'Psych Up', 'Hello Nova', 'ClassBot', 'iCognito', 'AneChat', 'Talk2Alfie CIC',

,Company Names (Leave Blank if Using LinkedIn Company Mining)
0,Woebot Health
1,Wysa
2,Plutus Health Inc.
3,Julian Jewel's AI Bot
4,X2AI
...,...
243,Ascent
244,myFolks Technology
245,"Medical Cyberworlds, Inc."
246,Red Ant Digital Marketing
